http://cmusatyalab.github.io/openface/

In [3]:
!ls

align-dlib_mod.py	       landmarkCatDog
catDog_test		       landmark_catDog.zip
cat_dog_main.ipynb	       landmarks.ipynb
cat_dog_report.ipynb	       landmarks_cat_dog.ipynb
chopHead256_real_soyoung_pics  newFlask
chop_code		       opencvTest.ipynb
facial_landmarks.py	       openface.ipynb
facial_landmarks2.py	       pic_report.ipynb
freelencer_face.ipynb	       real_soyoung_pics
headLab			       shape_predictor_68_face_landmarks.dat
idFace.ipynb		       soyoung_test
lab


In [2]:
!pwd

/home/face


#### training files in total

In [ ]:
for N in {1..8}; do ./util/align-dlib.py data/lfw/raw align outerEyesAndNose data/lfw/dlib-affine-sz:96 --size 96 & done

### 1. generate chopped head 

#### head size 256*256

In [21]:
%%time
!for N in {1..3}; do /root/openface/util/align-dlib.py \
./catVsdog/ align outerEyesAndNose ./catDog_Head_256 --size 256 & done

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 763 ms


In [2]:
!date

Fri Sep  8 23:39:12 UTC 2017


#### lost how many (not detect faces in pic)

#### 256

In [26]:
%%time
! /root/openface/batch-represent/main.lua \
-outDir catDog_data_256 -data catDog_Head_256

{
  data : "catDog_Head_256"
  imgDim : 96
  model : "/root/openface/models/openface/nn4.small2.v1.t7"
  device : 1
  outDir : "catDog_data_256"
  cache : false
  cuda : false
  batchSize : 50
}
catDog_Head_256	
cache lotation: 	/home/face/catDog_Head_256/cache.t7	
Creating metadata for cache.	
{
  sampleSize : 
    {
      1 : 3
      2 : 96
      3 : 96
    }
  split : 0
  verbose : true
  paths : 
    {
      1 : "catDog_Head_256"
    }
  samplingMode : "balanced"
  loadSize : 
    {
      1 : 3
      2 : 96
      3 : 96
    }
}
running "find" on each class directory, and concatenate all those filenames into a single file containing all image paths for a given class	
now combine all the files to a single large file	
load the large concatenated list of sample paths to self.imagePath	
450 samples found.	.....................]                              ETA: 0ms | Step: 0m 0/450 
Updating classList and imageClass appropriately	
 [=============================================>]       

#### check feature file dimentions

In [27]:
import pandas as pd

In [46]:
f256=pd.read_csv('catDog_data_256/reps.csv', header=None)
f256.shape

(450, 128)

In [47]:
f256.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,
0,0.084256,0.124602,-0.032911,-0.005287,-0.062790,0.158262,0.043912,0.082694,0.086530,-0.132431,-0.003451,0.028661,0.013015,-0.112457,-0.057183,-0.056525,-0.093483,-0.171704,-0.197998,0.021992,...
1,0.145251,0.102179,-0.159995,0.091649,0.098659,0.052362,0.059707,-0.089451,-0.018852,-0.120745,0.071557,0.036485,0.083100,-0.054829,-0.092525,0.039074,0.005904,0.059043,-0.193330,0.161276,...
2,0.028025,0.036355,-0.047401,-0.067069,0.103842,-0.002339,0.012883,0.079872,0.127512,-0.113367,-0.029214,0.070639,0.061048,-0.061276,-0.050789,-0.004422,-0.033777,-0.026529,-0.313481,0.109807,...
3,0.066869,0.002742,-0.032741,0.078806,0.027590,-0.016663,0.033716,0.039150,0.065654,-0.131226,0.021373,0.152363,-0.000653,-0.085730,-0.082067,0.003570,-0.019690,-0.142042,-0.174617,0.162027,...
4,0.124326,0.057964,-0.089978,0.062997,0.010768,0.107759,0.082306,0.025259,0.089133,-0.102017,0.042654,0.072407,0.034893,-0.206650,-0.140678,0.035104,-0.087567,-0.101625,-0.259181,0.142993,...


#### train classifier using only SVM in scikit-learn

In [51]:
%%time
!/root/openface/demos/classifier.py train catDog_data_256

/root/.local/lib/python2.7/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
Loading embeddings.
Training for 2 classes.
Saving classifier to 'catDog_data_256/classifier.pkl'
CPU times: user 28 ms, sys: 32 ms, total: 60 ms
Wall time: 2.38 s


### make prediction on tests , un-train pics 

### note: out of 60 test pics (30 from each), 21 out of 60 cannot identify head,  yet for the 39 pics with head detected, only 2 misclassification. 2/39 

### PS: images saved as 256x256 has better confidence than 96x96

96

In [55]:
%%time
!/root/openface/demos/classifier.py \
infer ./catDog_data_192/classifier.pkl \
test_catDog/*

/root/.local/lib/python2.7/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)

=== test_catDog/110814_0.jpg ===
Predict dog with 0.94 confidence.

=== test_catDog/110814_104.jpg ===
Predict dog with 0.99 confidence.

=== test_catDog/110814_118.jpg ===
cannot find even one face in the picture

=== test_catDog/110814_133.jpg ===
Predict cat with 0.64 confidence.

=== test_catDog/110814_152.jpg ===
Predict dog with 1.00 confidence.

=== test_catDog/110814_163.jpg ===
Predict dog with 0.88 confidence.

=== test_catDog/110814_18.jpg ===
cannot find even one face in the picture

=== test_catDog/110814_181.jpg ===
cannot find even one face in the picture

=== test_catDog/110814_211.jpg ===
Predict dog with 1.00 confidence.

=== test_catDog/110814_221.jpg ===
cannot find even one face in the picture

=== test_catDog/110814_

256

In [56]:
%%time
!/root/openface/demos/classifier.py \
infer ./catDog_data_256/classifier.pkl \
test_catDog/*

/root/.local/lib/python2.7/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)

=== test_catDog/110814_0.jpg ===
Predict dog with 0.93 confidence.

=== test_catDog/110814_104.jpg ===
Predict dog with 0.99 confidence.

=== test_catDog/110814_118.jpg ===
cannot find even one face in the picture

=== test_catDog/110814_133.jpg ===
Predict cat with 0.67 confidence.

=== test_catDog/110814_152.jpg ===
Predict dog with 1.00 confidence.

=== test_catDog/110814_163.jpg ===
Predict dog with 0.88 confidence.

=== test_catDog/110814_18.jpg ===
cannot find even one face in the picture

=== test_catDog/110814_181.jpg ===
cannot find even one face in the picture

=== test_catDog/110814_211.jpg ===
Predict dog with 1.00 confidence.

=== test_catDog/110814_221.jpg ===
cannot find even one face in the picture

=== test_catDog/110814_